# VacationPy
---
#### Note
- Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import requests
import pprint as pp

# Import API key
from config import g_key


## Store Part I results into DataFrame
- Load the csv exported in Part I to a Dataframe

In [2]:
cities_df = pd.read_csv('../output_data/cities.csv')
# fix index column name
cities_df = cities_df.rename(columns={'Unnamed: 0':''})
cities_df = cities_df.set_index('')
cities_df.head()

,city,country,id,date,latitude,longitude,temp,cloudiness,wind speed,humidity
,,,,,,,,,,
0,Beloha,MG,1067565,09/26/19 18:07,-25.17,45.06,67.4654,82,8.97,78
1,Shikhazany,RU,494514,09/26/19 18:07,55.56,47.39,39.2000,0,5.00,93
2,Butaritari,KI,2110227,09/26/19 18:07,3.07,172.79,83.7014,43,5.25,74
3,Thompson,CA,6165406,09/26/19 18:07,55.74,-97.86,42.8000,75,4.60,48
4,Mecca,SA,104515,09/26/19 18:07,21.43,39.83,82.2254,89,2.12,47


In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
# Create a Gmaps figure
fig = gmaps.figure()

locations = cities_df[['latitude','longitude']]
weights = cities_df['humidity']
# Create the Heat Layer Object 
heat_layer = gmaps.heatmap_layer(locations=locations, weights=weights, dissipating=True)
heat_layer.point_radius = 10
# Add Heat Layer to Figure
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Calculate ideal values for temp, cloudiness, windspeed, humidity
min_temp = cities_df['temp']>70 
max_temp = cities_df['temp']<80
max_cloud = cities_df['cloudiness'] < 30
max_wind = cities_df['wind speed'] < 4.00
max_humidity = cities_df['humidity'] < 70

# Set it to a new DataFrame
ideal_weather = cities_df.where(min_temp & max_temp & max_cloud & max_wind & max_humidity)
ideal_weather.dropna(inplace=True)
ideal_weather

,city,country,id,date,latitude,longitude,temp,cloudiness,wind speed,humidity
,,,,,,,,,,
24,Basco,US,4863349.0,09/26/19 18:07,40.33,-91.20,73.6880,1.0,2.10,33.0
232,Independence,US,4391812.0,09/26/19 18:04,39.09,-94.42,76.0280,1.0,2.60,31.0
242,San Patricio,PY,3437029.0,09/26/19 18:07,-26.98,-56.83,73.9814,0.0,2.20,50.0
361,Pleasanton,US,5383777.0,09/26/19 18:06,37.66,-121.87,79.5380,1.0,3.60,39.0
371,Nizwa,OM,286987.0,09/26/19 18:08,22.93,57.53,73.5134,1.0,0.57,69.0
411,Price,US,5545269.0,09/26/19 18:06,39.60,-110.81,78.9620,1.0,3.72,19.0
448,Miajadas,ES,2513893.0,09/26/19 18:08,39.15,-5.91,71.0060,0.0,1.34,43.0
472,Trairi,BR,3386177.0,09/26/19 18:08,-3.28,-39.27,76.7174,0.0,2.76,68.0
499,San Juan,AR,3837213.0,09/26/19 18:07,-31.54,-68.53,71.0114,0.0,3.41,22.0


In [16]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}


hotel_lists = []
hotel_markers = []

# Use the lat/lng we recovered to identify airports
for index, row in ideal_weather.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)    
    # convert to json
    name_address = name_address.json()

    # Grab the first hit and update hotel_data dict
    if name_address['results']:
        first_hit = name_address['results'][0]
        hotel_lists.append(first_hit['name'])
        hotel_markers.append((first_hit['geometry']['location']['lat'],first_hit['geometry']['location']['lng']))
    else:
        hotel_lists.append(np.nan)

In [17]:
hotel_lists

[nan,
 'Higher Ground Hotel',
 nan,
 'The Rose Hotel',
 'Nizwa Residence Hotel Apartments',
 'Ramada by Wyndham Price',
 'Hotel Triana',
 'Rede Beach: Hotel em Guajiru',
 'Hotel Alkazar',
 'Holiday Inn Express State College @Williamsburg Sq',
 'Akhavan Hotel']

### Hotel Map
- Store into variable named hotel_df.
- Add a "Hotel Name" column to the DataFrame.
- Set parameters to search for hotels with 5000 meters.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.
- Plot markers on top of the heatmap.

In [22]:
hotel_df = ideal_weather
hotel_df['hotel'] = hotel_lists
hotel_df = hotel_df.dropna()
hotel_df

,city,country,id,date,latitude,longitude,temp,cloudiness,wind speed,humidity,hotel
,,,,,,,,,,,
232,Independence,US,4391812.0,09/26/19 18:04,39.09,-94.42,76.0280,1.0,2.60,31.0,Higher Ground Hotel
361,Pleasanton,US,5383777.0,09/26/19 18:06,37.66,-121.87,79.5380,1.0,3.60,39.0,The Rose Hotel
371,Nizwa,OM,286987.0,09/26/19 18:08,22.93,57.53,73.5134,1.0,0.57,69.0,Nizwa Residence Hotel Apartments
411,Price,US,5545269.0,09/26/19 18:06,39.60,-110.81,78.9620,1.0,3.72,19.0,Ramada by Wyndham Price
448,Miajadas,ES,2513893.0,09/26/19 18:08,39.15,-5.91,71.0060,0.0,1.34,43.0,Hotel Triana
472,Trairi,BR,3386177.0,09/26/19 18:08,-3.28,-39.27,76.7174,0.0,2.76,68.0,Rede Beach: Hotel em Guajiru
499,San Juan,AR,3837213.0,09/26/19 18:07,-31.54,-68.53,71.0114,0.0,3.41,22.0,Hotel Alkazar
513,State College,US,5213681.0,09/26/19 18:08,40.79,-77.86,71.0600,1.0,2.60,53.0,Holiday Inn Express State College @Williamsbur...
570,Kerman,IR,128234.0,09/26/19 18:08,30.29,57.06,71.6000,20.0,1.00,6.0,Akhavan Hotel


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_markers

In [31]:
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))